## **Importing necessary Libraries** 

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import xgboost
from imblearn.combine import SMOTETomek,SMOTEENN
from collections import Counter
import lightgbm
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
!pip install catboost
import catboost
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold,cross_val_score

## **Reading the Data**

In [175]:
df=pd.read_csv('/content/train_MpHjUjU.csv')

In [176]:
df.head(10)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1
7,2017-02-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1
8,2017-03-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,350000,1
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1


## **As There was no Target column hence we had to find some metrics to decide the Target,According to my knowledge i have grouped the data by sum of Total Business value for each emp_id,if the emp_id has 0 Total business value i have labeled it as 1 i'e he leaves the comapany else 0**

In [177]:
df1=df.groupby(['Emp_ID'])[['Total Business Value']].sum().reset_index()

In [178]:
s=df1[df1['Total Business Value']==0].Emp_ID.values

In [179]:
df['Target']=df['Emp_ID'].map(lambda x:1 if x in s else 0)

In [180]:
df['Target'].value_counts()  # Imbalanced Dataset

0    17004
1     2100
Name: Target, dtype: int64

In [181]:
df=df.drop(['LastWorkingDate'],axis=1) # LastWorkingDate id dropped as it had majority of null values

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MMM-YY                19104 non-null  object
 1   Emp_ID                19104 non-null  int64 
 2   Age                   19104 non-null  int64 
 3   Gender                19104 non-null  object
 4   City                  19104 non-null  object
 5   Education_Level       19104 non-null  object
 6   Salary                19104 non-null  int64 
 7   Dateofjoining         19104 non-null  object
 8   Joining Designation   19104 non-null  int64 
 9   Designation           19104 non-null  int64 
 10  Total Business Value  19104 non-null  int64 
 11  Quarterly Rating      19104 non-null  int64 
 12  Target                19104 non-null  int64 
dtypes: int64(8), object(5)
memory usage: 1.9+ MB


## **As training and Testing data was present in single train.csv so we had to differentiat test records from train.csv based on given emp_id in test.csv**

In [183]:
test=pd.read_csv('/content/test_hXY9mYw.csv')

In [184]:
ss=[i for i in test['Emp_ID']]

In [185]:
df['Test']=df['Emp_ID'].apply(lambda x:1 if x in ss else 0)

In [186]:
test=df[df['Test']==1]

In [187]:
train=df[df['Test']==0]

In [188]:
train.drop('Test',axis=1,inplace=True)
test.drop('Test',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [189]:
test

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,2,2,0,1,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,2,2,0,1,1
13,2017-08-01,6,31,Female,C11,Bachelor,78728,2017-07-31,3,3,0,1,0
14,2017-09-01,6,31,Female,C11,Bachelor,78728,2017-07-31,3,3,0,1,0
15,2017-10-01,6,31,Female,C11,Bachelor,78728,2017-07-31,3,3,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2017-08-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,740280,3,0
19100,2017-09-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,448370,3,0
19101,2017-10-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,0,2,0
19102,2017-11-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,200420,2,0


In [190]:
test1=pd.read_csv('/content/test_hXY9mYw.csv')
test1.head()

,Emp_ID
0,394
1,173
2,1090
3,840
4,308


In [191]:
test1['Target']=test1['Emp_ID'].map(lambda x:test[test['Emp_ID']==x]['Target'].unique()[0])

In [192]:
train['Target'].value_counts()

0    8725
1    1791
Name: Target, dtype: int64

## **Feature Enginerring i have converted the dateof join and MMM-YY column to datetime object and then created features such as day,month,year,weekday,day_of_Week**

In [193]:
train['MMM-YY']=pd.to_datetime(train['MMM-YY'])
train['Dateofjoining']=pd.to_datetime(train['Dateofjoining'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [194]:
train['MMM-YY_day']=train['MMM-YY'].dt.day
train['MMM-YY_month']=train['MMM-YY'].dt.month 
train['MMM-YY_year']=train['MMM-YY'].dt.year
train['MMM-YY_week']=train['MMM-YY'].dt.weekday
train['MMM-YY_dayofweek']=train['MMM-YY'].dt.dayofweek
train['Dateofjoining_day']=train['Dateofjoining'].dt.day 
train['Dateofjoining_month']=train['Dateofjoining'].dt.month 
train['Dateofjoining_year']=train['Dateofjoining'].dt.year
train['Dateofjoining_week']=train['Dateofjoining'].dt.weekday
train['Dateofjoining_dayofweek']=train['Dateofjoining'].dt.dayofweek

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [195]:
train_1

,Emp_ID,Age,City,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,MMM-YY_day,MMM-YY_month,MMM-YY_year,MMM-YY_week,MMM-YY_dayofweek,Dateofjoining_day,Dateofjoining_month,Dateofjoining_year,Dateofjoining_week,Dateofjoining_dayofweek,Gender_Male,Education_Level_College,Education_Level_Master
0,1,28,23,57387,1,1,2381060,2,0,1,1,2016,4,4,24,12,2015,3,3,1,0,1
1,1,28,23,57387,1,1,-665480,2,0,1,2,2016,0,0,24,12,2015,3,3,1,0,1
2,1,28,23,57387,1,1,0,2,0,1,3,2016,1,1,24,12,2015,3,3,1,0,1
5,4,43,13,65603,2,2,0,1,0,1,12,2016,3,3,7,12,2016,2,2,1,0,1
6,4,43,13,65603,2,2,0,1,0,1,1,2017,6,6,7,12,2016,2,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19092,2787,28,20,69498,1,1,250000,2,0,1,2,2016,0,0,21,7,2015,1,1,0,0,1
19093,2787,28,20,69498,1,1,319740,2,0,1,3,2016,1,1,21,7,2015,1,1,0,0,1
19094,2787,28,20,69498,1,1,0,1,0,1,4,2016,4,4,21,7,2015,1,1,0,0,1
19095,2787,28,20,69498,1,1,0,1,0,1,5,2016,6,6,21,7,2015,1,1,0,0,1


## **Emp_id Was dropped before training the model**

In [196]:
train_1=train.drop(['MMM-YY','Dateofjoining','Emp_ID'],axis=1)

In [197]:
train_1

,Age,Gender,City,Education_Level,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,MMM-YY_day,MMM-YY_month,MMM-YY_year,MMM-YY_week,MMM-YY_dayofweek,Dateofjoining_day,Dateofjoining_month,Dateofjoining_year,Dateofjoining_week,Dateofjoining_dayofweek
0,28,Male,C23,Master,57387,1,1,2381060,2,0,1,1,2016,4,4,24,12,2015,3,3
1,28,Male,C23,Master,57387,1,1,-665480,2,0,1,2,2016,0,0,24,12,2015,3,3
2,28,Male,C23,Master,57387,1,1,0,2,0,1,3,2016,1,1,24,12,2015,3,3
5,43,Male,C13,Master,65603,2,2,0,1,0,1,12,2016,3,3,7,12,2016,2,2
6,43,Male,C13,Master,65603,2,2,0,1,0,1,1,2017,6,6,7,12,2016,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19092,28,Female,C20,Master,69498,1,1,250000,2,0,1,2,2016,0,0,21,7,2015,1,1
19093,28,Female,C20,Master,69498,1,1,319740,2,0,1,3,2016,1,1,21,7,2015,1,1
19094,28,Female,C20,Master,69498,1,1,0,1,0,1,4,2016,4,4,21,7,2015,1,1
19095,28,Female,C20,Master,69498,1,1,0,1,0,1,5,2016,6,6,21,7,2015,1,1


## **Converting Categorical columns to numerical columns using encoding techniques like one hot Encoding,also we can try label encoding for education as it is ordinal**

In [199]:
train_1['City']=train_1['City'].apply(lambda x:x.lstrip('C'))

In [200]:
train_1['City']=train_1['City'].astype(int)

In [201]:
train_1=pd.get_dummies(train_1,drop_first=True)

In [202]:
train_1['Target'].value_counts()

0    8725
1    1791
Name: Target, dtype: int64

In [229]:
train_1

,Age,City,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,MMM-YY_day,MMM-YY_month,MMM-YY_year,MMM-YY_week,MMM-YY_dayofweek,Dateofjoining_day,Dateofjoining_month,Dateofjoining_year,Dateofjoining_week,Dateofjoining_dayofweek,Gender_Male,Education_Level_College,Education_Level_Master
0,28,23,57387,1,1,2381060,2,0,1,1,2016,4,4,24,12,2015,3,3,1,0,1
1,28,23,57387,1,1,-665480,2,0,1,2,2016,0,0,24,12,2015,3,3,1,0,1
2,28,23,57387,1,1,0,2,0,1,3,2016,1,1,24,12,2015,3,3,1,0,1
5,43,13,65603,2,2,0,1,0,1,12,2016,3,3,7,12,2016,2,2,1,0,1
6,43,13,65603,2,2,0,1,0,1,1,2017,6,6,7,12,2016,2,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19092,28,20,69498,1,1,250000,2,0,1,2,2016,0,0,21,7,2015,1,1,0,0,1
19093,28,20,69498,1,1,319740,2,0,1,3,2016,1,1,21,7,2015,1,1,0,0,1
19094,28,20,69498,1,1,0,1,0,1,4,2016,4,4,21,7,2015,1,1,0,0,1
19095,28,20,69498,1,1,0,1,0,1,5,2016,6,6,21,7,2015,1,1,0,0,1


## **Various techniques like Smote and smotetomek were tried out to balance the data unfortunately it did not give good results.**

In [346]:
sm=SMOTETomek(random_state=139)

In [350]:
x_train_sm,y_train_sm=sm.fit_resample(X_train,y_train)

In [349]:
x_train_sms,y_train_sms=sm.fit_resample(X,y)

## **Splitting the Data into Train and test**

In [203]:
X=train_1.drop('Target',axis=1)
y=train_1['Target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [287]:
models={'Gradient':GradientBoostingClassifier(),'xgboost':xgboost.XGBClassifier(),'Xgbr':xgboost.XGBRFClassifier(max_depth=4),'rd':RandomForestClassifier(),'dt':DecisionTreeClassifier(),'cat':catboost.CatBoostClassifier()}

In [288]:
ds=pd.DataFrame(columns=[])
ri=0
for i in models:
  models[i].fit(X_train,y_train)
  ipred=models[i].predict(X_test)
  ipred_tr=models[i].predict(X_train)
  ds.loc[ri,'Model']=i
  ds.loc[ri,'Train_score']=f1_score(y_train,ipred_tr,average='macro')
  ds.loc[ri,'Test_score']=f1_score(y_test,ipred,average='macro')
  ri+=1

Learning rate set to 0.024161
0:	learn: 0.6488278	total: 3.35ms	remaining: 3.35s
1:	learn: 0.6080951	total: 6.56ms	remaining: 3.27s
2:	learn: 0.5712971	total: 9.65ms	remaining: 3.21s
3:	learn: 0.5407123	total: 12.9ms	remaining: 3.21s
4:	learn: 0.5131247	total: 16.2ms	remaining: 3.21s
5:	learn: 0.4879376	total: 19.6ms	remaining: 3.24s
6:	learn: 0.4653056	total: 22.8ms	remaining: 3.24s
7:	learn: 0.4436422	total: 26.5ms	remaining: 3.29s
8:	learn: 0.4247549	total: 29.8ms	remaining: 3.29s
9:	learn: 0.4071953	total: 33.2ms	remaining: 3.29s
10:	learn: 0.3924477	total: 36.5ms	remaining: 3.28s
11:	learn: 0.3794537	total: 40.5ms	remaining: 3.33s
12:	learn: 0.3680237	total: 44.1ms	remaining: 3.35s
13:	learn: 0.3593163	total: 47.3ms	remaining: 3.33s
14:	learn: 0.3509276	total: 51ms	remaining: 3.35s
15:	learn: 0.3438647	total: 54.3ms	remaining: 3.34s
16:	learn: 0.3395835	total: 57.8ms	remaining: 3.34s
17:	learn: 0.3318950	total: 61.2ms	remaining: 3.34s
18:	learn: 0.3289817	total: 64.6ms	remaining: 

## **Among ALL the models most generalized model with no overfitting was found to be xgbrf(Xgboost+Random_forest) so we have selected xgbr as final model for hyperparameter optimization**

In [289]:
ds

,Model,Train_score,Test_score
0,Gradient,0.867673,0.813774
1,xgboost,0.858120,0.809236
2,Xgbr,0.742689,0.739334
3,rd,1.000000,0.927285
4,dt,1.000000,0.891794
5,cat,0.989677,0.910382


## **Hyperparmeter Tuning and Cross Validataion**

In [318]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
 "scale_pos_weight" : [6,7,8,9,10],
 "n_estimators"      : [i for i in range(0,1000,100)]
    
}

In [333]:
st=StratifiedKFold(n_splits=5)
model=xgboost.XGBRFClassifier()
scores=cross_val_score(model,X,y,cv=st)
print(np.mean(scores))

0.8516545257634848


In [321]:
xgbr=xgboost.XGBRFClassifier()

In [403]:
random_search=RandomizedSearchCV(xgbr,param_distributions=params,n_iter=5,scoring='f1_macro',n_jobs=-1,cv=3,verbose=3)

In [404]:
random_search.fit(x_train_sms,y_train_sms)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=XGBRFClassifier(max_depth=5), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [0, 100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900],
                                        'scale_pos_weight': [6, 7, 8, 9, 10]},
                 

In [405]:
random_search.best_params_

{'colsample_bytree': 0.7,
 'gamma': 0.4,
 'learning_rate': 0.15,
 'max_depth': 3,
 'min_child_weight': 7,
 'n_estimators': 700,
 'scale_pos_weight': 10}

In [407]:
xgbr_hyper=xgboost.XGBRFClassifier(n_estimators=700,colsample_bytree=0.7,gamma=0.4,learning_rate=0.15,max_dept=3,min_child_weight=7,scale_pos_weight=10)

In [408]:
xgbr_hyper.fit(x_train_sm,y_train_sm)
xgbr_hyperpred=xgbr_hyper.predict(X_test)
print(f"{'Train'}:{f1_score(y_train,xgbr_hyper.predict(X_train),average='macro')}")
print(f"{'Test'}:{f1_score(y_test,xgbr_hyperpred,average='macro')}")

Train:0.7016969758370902
Test:0.6933399860034732


In [410]:
xgbr_hyper.fit(x_train_sms,y_train_sms)

XGBRFClassifier(colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_dept=3,
                min_child_weight=7, n_estimators=700, scale_pos_weight=10)

## **Tried out without hyperpameter tuning by selecting max_depth as 5 as it was giving good results in terms of training and testing data**

In [419]:
xgbr=xgboost.XGBRFClassifier(max_depth=5)
xgbr.fit(X_train,y_train)
xgbr_pred=xgbr.predict(X_test)
print(f1_score(y_train,xgbr.predict(X_train),average='macro'))
print(f1_score(y_test,xgbr_pred,average='macro'))

0.7703970478146533
0.7563021291045635


In [420]:
xgbr.fit(X,y)

XGBRFClassifier(max_depth=5)

## **Repeating all the preprocessing Steps for Test data**

In [211]:
test['MMM-YY']=pd.to_datetime(test['MMM-YY'])
test['Dateofjoining']=pd.to_datetime(test['Dateofjoining'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [212]:
test['MMM-YY_day']=test['MMM-YY'].dt.day
test['MMM-YY_month']=test['MMM-YY'].dt.month 
test['MMM-YY_year']=test['MMM-YY'].dt.year
test['MMM-YY_week']=test['MMM-YY'].dt.weekday
test['MMM-YY_dayofweek']=test['MMM-YY'].dt.dayofweek
test['Dateofjoining_day']=test['Dateofjoining'].dt.day 
test['Dateofjoining_month']=test['Dateofjoining'].dt.month 
test['Dateofjoining_year']=test['Dateofjoining'].dt.year
test['Dateofjoining_week']=test['Dateofjoining'].dt.weekday
test['Dateofjoining_dayofweek']=test['Dateofjoining'].dt.dayofweek

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [244]:
test

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,MMM-YY_day,MMM-YY_month,MMM-YY_year,MMM-YY_week,MMM-YY_dayofweek,Dateofjoining_day,Dateofjoining_month,Dateofjoining_year,Dateofjoining_week,Dateofjoining_dayofweek
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,2,2,0,1,1,1,11,2017,2,2,6,11,2017,0,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,2,2,0,1,1,1,12,2017,4,4,6,11,2017,0,0
13,2017-08-01,6,31,Female,C11,Bachelor,78728,2017-07-31,3,3,0,1,0,1,8,2017,1,1,31,7,2017,0,0
14,2017-09-01,6,31,Female,C11,Bachelor,78728,2017-07-31,3,3,0,1,0,1,9,2017,4,4,31,7,2017,0,0
15,2017-10-01,6,31,Female,C11,Bachelor,78728,2017-07-31,3,3,0,2,0,1,10,2017,6,6,31,7,2017,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2017-08-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,740280,3,0,1,8,2017,1,1,8,6,2017,3,3
19100,2017-09-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,448370,3,0,1,9,2017,4,4,8,6,2017,3,3
19101,2017-10-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,0,2,0,1,10,2017,6,6,8,6,2017,3,3
19102,2017-11-01,2788,30,Male,C27,Master,70254,2017-06-08,2,2,200420,2,0,1,11,2017,2,2,8,6,2017,3,3


In [247]:
test_1=test.drop(['MMM-YY','Dateofjoining','Emp_ID'],axis=1)

In [248]:
test_1['City']=test_1['City'].apply(lambda x:x.lstrip('C'))

In [249]:
test_1['City']=test_1['City'].astype(int)

In [250]:
test_1=pd.get_dummies(test_1,drop_first=True)

In [252]:
test_1.drop(['Target'],axis=1,inplace=True)

In [253]:
test_1

,Age,City,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,MMM-YY_day,MMM-YY_month,MMM-YY_year,MMM-YY_week,MMM-YY_dayofweek,Dateofjoining_day,Dateofjoining_month,Dateofjoining_year,Dateofjoining_week,Dateofjoining_dayofweek,Gender_Male,Education_Level_College,Education_Level_Master
3,31,7,67016,2,2,0,1,1,11,2017,2,2,6,11,2017,0,0,1,0,1
4,31,7,67016,2,2,0,1,1,12,2017,4,4,6,11,2017,0,0,1,0,1
13,31,11,78728,3,3,0,1,1,8,2017,1,1,31,7,2017,0,0,0,0,0
14,31,11,78728,3,3,0,1,1,9,2017,4,4,31,7,2017,0,0,0,0,0
15,31,11,78728,3,3,0,2,1,10,2017,6,6,31,7,2017,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,30,27,70254,2,2,740280,3,1,8,2017,1,1,8,6,2017,3,3,1,0,1
19100,30,27,70254,2,2,448370,3,1,9,2017,4,4,8,6,2017,3,3,1,0,1
19101,30,27,70254,2,2,0,2,1,10,2017,6,6,8,6,2017,3,3,1,0,1
19102,30,27,70254,2,2,200420,2,1,11,2017,2,2,8,6,2017,3,3,1,0,1


# **prediction for test emp_id and storing results in csv in required format as given in submission.csv**

In [254]:
t=pd.read_csv('/content/test_hXY9mYw.csv')

In [258]:
result=pd.DataFrame(test['Emp_ID'])

In [421]:
result['Target']=xgbr_hyper.predict(test_1)

In [422]:
t['Target']=t['Emp_ID'].map(lambda x:result[result['Emp_ID']==x]['Target'].unique()[0])

In [423]:
t.to_csv('final.csv')

# **Future Scope: We can build Pipeline and can develop a web application using streamlit and can be deployed on aws EC2**